In [1]:
import Pkg
using Combinatorics
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `C:\Users\nsthtz\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...


In [2]:
using DelimitedFiles
#data = readdlm(joinpath("dummyMajor.csv"),';')
data = readdlm(joinpath("dummyMajorA#.csv"),';')
#data = readdlm(joinpath("dummyMajorMultiple.csv"),';')
#data = readdlm(joinpath("dummyMinor.csv"),',')
#data = readdlm(joinpath("dummyMinorA#.csv"),';')
data = round.(Int, data)

128×24 Array{Int64,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 ⋮              ⋮              ⋮         

# # Convert piano roll into timestepx12 tonal mapping


In [3]:
#

tonal_map = zeros(Int8, size(data, 2), 12)
i=1
data[:, 1]

for i in 1:size(data, 2)
    active_notes = findall(x->x==1, data[:,i])
    #println(active_notes)
    for note in active_notes
        tonal_map[i, note%12+1] = 1
    end
    #println(timestep_chords[i,:])
end

tonal_map

24×12 Array{Int8,2}:
 0  0  0  0  0  0  0  0  0  0  1  0
 1  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  1  0

# # Pattern matcher to identify the best-fit key and scale from existing notes

In [4]:
# Functions and Global

pattern = [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]

# Weight of core notes (1, 4, 5) doubled
majorpattern = [1, 0, 0.5, 0, 0.5, 1, 0, 1, 0, 0.5, 0, 0.5]
minorpattern = [0.5, 0, 1, 0, 1, 0.5, 0, 0.5, 0, 1, 0, 0.5]

Keys = Dict{Integer, String}(1 => "C", 2 => "C#", 3 => "D", 4 => "D#", 5 => "E", 6 => "F", 7 => "F#", 8 => "G", 9 => "G#", 10 => "A", 11 => "A#", 12 => "B")


#Score In: Tx12 Matrix, pattern. Out: Best position(s)
function score_key(matrix, pattern)
    counter = zeros(Int8, 1, 12)
    for i in 1:size(matrix, 2)
        counter[i] = sum(matrix[:,i])
    end
    println("Key Counter is ", counter)
    
    key_score = zeros(Int8, 12, 1)
    
    for i in 1:size(pattern, 1)
        for j in 1:size(pattern, 1)
            key_score[i] += circshift(pattern, i-1)[j]*counter[j]
        end
    end
    println("Key Score is ", key_score)
    return key_score
end

# Find maximum from list
function findmaximum(list)
    positions = []
    max_value = maximum(list)
    for (i, x) in enumerate(list)
        if x == max_value
            push!(positions, i)
        end
    end
    return positions
end

# Find an optimal scale
function score_tonality(matrix, positions)
    
    counter = zeros(Int8, 1, 12)
    for i in 1:size(matrix, 2)
        counter[i] = sum(matrix[:,i])
    end
    
    tonality_scores = []
    
    for position in positions
        counter_shifted = circshift(counter, (0,-(position-1)))
        println("Position Counter for ", position, " is ", counter_shifted)
        
        major_score = 0
        minor_score = 0
        
        for i in 1:size(counter_shifted, 2)
            major_score += majorpattern[i]*counter_shifted[i]
            minor_score += minorpattern[i]*counter_shifted[i]
        end

        if major_score > minor_score
            push!(tonality_scores, [position, major_score, 1])
        else
            push!(tonality_scores, [position, minor_score, 0])
        end
        
    end 
    
    return tonality_scores
end

score_tonality (generic function with 1 method)

In [5]:
# Score matrix after pattern
key_score_result = score_key(tonal_map, pattern)
# score_result = [9, 5, 18, 9, 23, 18, 8, 2, 7, 16, 12, 11]

# Find positions of maximum score
positions = findmaximum(key_score_result)
sort!(positions)

# # Use positions to find the best key

tonality_results = score_tonality(tonal_map, positions)

# [Position (1 = C), Score, Scale (1 = Major, 0 = Minor)]
best_key = [0, 0, 1]

# In case of tied scores, check each for the best scale
for result_group in tonality_results
    if best_key[2] < result_group[2]
        best_key[1] = result_group[1]
        best_key[2] = result_group[2]
        best_key[3] = result_group[3]
    end
end
    
#Save Key
println(best_key)
if best_key[3] == 1
    key = best_key[1]
    key_print = Keys[best_key[1]]
    scale = "Major"
else
    key = (12+best_key[1]-3)%12
    key_print = Keys[key]
    scale = "Minor"
end
    
println(mod(-1, 12))
    
println(key, " ", key_print, " ", scale)

Key Counter is Int8[2 0 2 5 0 3 0 3 0 3 6 0]
Key Score is Int8[13; 16; 8; 21; 8; 19; 14; 10; 19; 5; 24; 11]
Position Counter for 11 is Int8[6 0 2 0 2 5 0 3 0 3 0 3]
[11, 19, 1]
11
11 A# Major


# # Define patterns for chord functions relative to key

In [6]:
# Function entry: "Name" => ([notes], [Bass note], [Double/Triple notes], [Rhythmical Weight (All, Strong, Weak, Off)], [Chord Specific rules])

functions = Dict{String, Array{Int64, 1}}

if scale == "Major"
#Major  =  [1, 0, 2, 0, 3, 4, 0, 5, 0, 6, 0, 7]
#
    
    functions = Dict( "T"  =>  (circshift([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["A"]),
    "S"    =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["A"]),
    "D"    =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["A"]),
    "T-5"  =>  (circshift([1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["A"]),
    "S-5"  =>  (circshift([0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["A"]),
    "D-5"  =>  (circshift([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["A"]),
    "T/3"   => (circshift([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((4+(key-1))%12)+1, 3) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["A"]),
    "S/3"   => (circshift([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((9+(key-1))%12)+1, 3) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["A"]),
    "D/3"   => (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((11+(key-1))%12)+1, 3) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["A"]),
    "T/5"   => (circshift([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 5) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["W"]),
    "S/5"   => (circshift([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 5) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["W"]),
    "D/5"   => (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["W"]),
    "D64"  =>  (circshift([4, 0, 0, 0, 6, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S"]),
    "T64"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 0, 0, 6, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S"]),
    "T54"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 5, 0, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S"]),
    "D54"  =>  (circshift([4, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S"]),
    "D7"   =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ nothing ], ["A"]),
    "D7/3" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((11+(key-1))%12)+1, 3) ], [ nothing ], ["A"]),
    "D7/5" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ nothing ], ["W", "O"]),
    "D7/7" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((5+(key-1))%12)+1, 7) ], [ nothing ], ["A"]),
    "D7-1" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 0, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((5+(key-1))%12)+1, 7) ], ["A"]),
    "S65"  =>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ nothing ], ["A"]),
    "S65/6"=>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((2+(key-1))%12)+1, 6) ], [ nothing ], ["S"]),
    "S6"   =>  (circshift([0, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["A"]),
    "Ss"   =>  (circshift([0, 0, 1, 0, 0, 3, 0, 0, 0, 5, 0, 0], (key-1)), [ (((2+(key-1))%12)+1, 1) ], [ (((2+(key-1))%12)+1, 1), (((5+(key-1))%12)+1, 3) ], ["A"]),
    "Ts"   =>  (circshift([3, 0, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0], (key-1)), [ (((9+(key-1))%12)+1, 1) ], [ (key, 3), (((9+(key-1))%12)+1, 1) ], ["A"]),
    "Tm"   =>  (circshift([0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 5], (key-1)), [ (((4+(key-1))%12)+1, 1) ], [ (((4+(key-1))%12)+1, 1), (((7+(key-1))%12)+1, 3) ], ["A"]))

elseif scale == "Minor"
#Minor  =  [1, 0, 2, 3, 0, 4, 0, 5, 6, 0, 7, 0]        
        
    functions = Dict( "T"   =>  (circshift([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ key ], [ key ], ["A"]),
    "S"   =>   (circshift([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], (key-1)), [ ((5+(key-1))%12)+1 ]),
    "D"   =>   (circshift([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0], (key-1)), [ ((7+(key-1))%12)+1 ]),
    "T-5" =>   (circshift([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], (key-1)), [ key ]),
    "S-5" =>   (circshift([0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], (key-1)), [ ((5+(key-1))%12)+1 ]),
    "D-5" =>   (circshift([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0], (key-1)), [ ((7+(key-1))%12)+1 ]),
    "T/3"  =>  (circshift([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ ((3+(key-1))%12)+1 ]),
    "S/3"  =>  (circshift([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], (key-1)), [ ((8+(key-1))%12)+1 ]),
    "D/3"  =>  (circshift([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0], (key-1)), [ ((10+(key-1))%12)+1 ]),
    "T/5"  =>  (circshift([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ ((7+(key-1))%12)+1 ]),
    "S/5"  =>  (circshift([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], (key-1)), [ ((0+(key-1))%12)+1 ]),
    "D/5"  =>  (circshift([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0], (key-1)), [ ((2+(key-1))%12)+1 ]),
    "D64" =>   (circshift([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ ((7+(key-1))%12)+1 ]),
    "T64" =>   (circshift([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], (key-1)), [ ((0+(key-1))%12)+1 ]),
    "T54" =>   (circshift([1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0], (key-1)), [ ((0+(key-1))%12)+1 ]),
    "D54" =>   (circshift([1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ ((7+(key-1))%12)+1 ]),
    "D7"  =>   (circshift([0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0], (key-1)), [ ((7+(key-1))%12)+1 ]),
    "D7/3"  => (circshift([0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0], (key-1)), [ ((10+(key-1))%12)+1 ]),
    "D7/5"  => (circshift([0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0], (key-1)), [ ((2+(key-1))%12)+1 ]),
    "D7/7"  => (circshift([0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0], (key-1)), [ ((5+(key-1))%12)+1 ]),
    "D7-1" =>  (circshift([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0], (key-1)), [ ((2+(key-1))%12)+1 ]),
    "S65" =>   (circshift([1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0], (key-1)), [ ((5+(key-1))%12)+1 ]),
    "S65/6" => (circshift([1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0], (key-1)), [ ((2+(key-1))%12)+1 ]),
    "S6"  =>   (circshift([0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0], (key-1)), [ ((5+(key-1))%12)+1 ]),
    "Dm"  =>   (circshift([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0], (key-1)), [ ((10+(key-1))%12)+1 ]),
    "Ts"  =>   (circshift([1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0], (key-1)), [ ((8+(key-1))%12)+1 ]),
    "Tm"  =>   (circshift([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0], (key-1)), [ ((3+(key-1))%12)+1 ]))
        
end
        
for k in functions
    println(k[1], " ", k[2])
end

#      [C, C#, D, D#, E, F, F#, G, G#, A, A#, B]
#      [1, 2,  3, 4,  5, 6, 7,  8, 9, 10, 11, 12]
#      [2, 0,  3, 4,  0, 5, 0,  6, 0, 7,  1,  0]

S65/6 ([6, 0, 0, 1, 0, 0, 0, 3, 0, 0, 5, 0], Tuple{Int64,Int64}[(1, 6)], Nothing[nothing], ["S"])
S/5 ([0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 5, 0], Tuple{Int64,Int64}[(11, 5)], Tuple{Int64,Int64}[(11, 5), (4, 1)], ["W"])
T ([0, 0, 3, 0, 0, 5, 0, 0, 0, 0, 1, 0], Tuple{Int64,Int64}[(11, 1)], Tuple{Int64,Int64}[(11, 1)], ["A"])
Ts ([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 3, 0], Tuple{Int64,Int64}[(8, 1)], Tuple{Int64,Int64}[(11, 3), (8, 1)], ["A"])
S/3 ([0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 5, 0], Tuple{Int64,Int64}[(8, 3)], Tuple{Int64,Int64}[(11, 5), (4, 1)], ["A"])
T/5 ([0, 0, 3, 0, 0, 5, 0, 0, 0, 0, 1, 0], Tuple{Int64,Int64}[(6, 5)], Tuple{Int64,Int64}[(11, 1), (6, 5)], ["W"])
Ss ([1, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 0], Tuple{Int64,Int64}[(1, 1)], Tuple{Int64,Int64}[(1, 1), (4, 3)], ["A"])
D7/5 ([5, 0, 0, 7, 0, 1, 0, 0, 0, 3, 0, 0], Tuple{Int64,Int64}[(1, 5)], Nothing[nothing], ["W", "O"])
T/3 ([0, 0, 3, 0, 0, 5, 0, 0, 0, 0, 1, 0], Tuple{Int64,Int64}[(3, 3)], Tuple{Int64,Int64}[(11, 1), (6, 5)], ["A"])
D/5 ([

# # Analyze tonal map and calculate hamming distances for each timestep

In [7]:
##### DISTANCE FUNCTIONS

function hamming(input, target)
    counter = 0
    for i in 1:size(input, 1)
        if input[i] != target[i]
            counter = counter+1
        end
    end
    return counter
end

## What is the distance of finding a valid function within the input
        
function distance(input, target)
    distance = 0
    input_padded = append!(append!(input, input), input)[1:36]
    for i in 1:size(target, 1)
        if target[i] == 1
            if input[i] != 1
                for j in 1:6
                    if input_padded[12+i-j] == 1
                        distance += j
                        break
                    elseif input_padded[12+i+j] == 1
                        distance += j
                        break
                    end
                end
            end
        end
    end
    return distance
end
        
## What is the distance to transform an input into a function
    
function distance(input, target)
    distance = 0
    target_padded = copy(target)
    target_padded = append!(append!(target_padded, target), target)[1:36]
    for i in 1:size(input, 1)
        if input[i] > 0
            if target[i] > 0
                for x in 0:2
                    target_padded[i+(12*x)] = 0
                end
            else
                for j in 1:6
                    if target_padded[12+i-j] > 0
                        distance += j
                        break
                    elseif target_padded[12+i+j] > 0
                        distance += j
                        break
                    end
                end
            end
        end
    end
    
    return distance
end
                


####

chords = keys(functions)
function_potential = []

for i in 1:size(tonal_map, 1)         
    #println(tonal_map[i,:])
    function_pool = []
    for chord in chords
        hamming_distance = distance(tonal_map[i,:], functions[chord][1])
        push!(function_pool, (hamming_distance, chord))
    end
    push!(function_potential, sort(function_pool))
end

println(function_potential[1])

## Naive function label
                    
chosen_chords = []                    
                  
    
for timestep in 1:size(function_potential, 1)
    append!(chosen_chords, [function_potential[timestep][1][2]])
end
                    
println(chosen_chords)


Any[(0, "D54"), (0, "D64"), (0, "S"), (0, "S/3"), (0, "S/5"), (0, "S65"), (0, "S65/6"), (0, "T"), (0, "T-5"), (0, "T/3"), (0, "T/5"), (0, "T54"), (0, "T64"), (0, "Ts"), (1, "D"), (1, "D-5"), (1, "D/3"), (1, "D/5"), (1, "D7"), (1, "D7-1"), (1, "D7/3"), (1, "D7/5"), (1, "D7/7"), (1, "Tm"), (2, "S6"), (2, "Ss"), (3, "S-5")]
Any["D54", "D", "D64", "D7", "D", "S", "D", "D54", "D54", "D", "S", "D", "D7", "D64", "D", "D54", "D7", "D54", "D7", "D", "D7", "S", "D", "D54"]


# # Construct a naive solution

In [8]:
function getNotes(timestep_vector)
    return findall(x->x==1, timestep_vector)
end

function voicify(original_timestep_notes)
    if size(original_timestep_notes, 1) > 4
        println("yes")
    el
    end
    #return voicing
end

# # Note: Only hamming distances are considered from the original work

function convertTonalMapToNotes(tonal_Map, voice="X")
    
    indexes = getNotes(tonal_map)
    indexes = indexes .- 1
    
    if voice == "B"
        notes = [x for x in 40:61 if x%12 in indexes]
    elseif voice == "T"
        notes = [x for x in 47:68 if x%12 in indexes]
    elseif voice == "A"
        notes = [x for x in 52:73 if x%12 in indexes]
    elseif voice == "S"
        notes = [x for x in 59:80 if x%12 in indexes]
    else
        notes = []
        push!(notes, [x for x in 40:61 if x%12 in indexes], [x for x in 47:68 if x%12 in indexes], [x for x in 52:73 if x%12 in indexes], [x for x in 59:80 if x%12 in indexes])
    end
    
    return notes
end

#println(convertTonalMapToNotes(functions["T-5"][1], "X"))

function convertFunctionToNotes(func, voice="X")
    
    indexes = getNotes(functions[func][1])
    indexes = indexes .- 1
    
    println(functions[func][3])
                    
    if voice == "B"
        notes = [x for x in 40:61 if x%12 == functions[func][3][1]-1]
    elseif voice == "T"
        notes = [x for x in 47:68 if x%12 in indexes]
    elseif voice == "A"
        notes = [x for x in 52:73 if x%12 in indexes]
    elseif voice == "S"
        notes = [x for x in 59:80 if x%12 in indexes]
    else
        notes = []
        push!(notes, [x for x in 40:61 if x%12 in functions[func][3][1]-1], [x for x in 47:68 if x%12 in indexes], [x for x in 52:73 if x%12 in indexes], [x for x in 59:80 if x%12 in indexes])
    end
    
    return notes
                    
end
     
println(convertFunctionToNotes("T-5", "X"))
                
#########

original_notes = []

for timestep_index in 1:size(data, 2)
    timestep_vector = data[:, timestep_index]
    original_notes_timestep = getNotes(timestep_vector)
    push!(original_notes, original_notes_timestep)
end

println(original_notes[1], " ", function_map[1][1])




##              1      2     3      4
## voices = [Bass, Tenor, Alto, Soprano]

[1]
Any[[48, 60], [48, 52, 60, 64], [52, 60, 64, 72], [60, 64, 72, 76]]


UndefVarError: UndefVarError: function_map not defined

# # Construct the search space for note distribution to voices

In [8]:
# Finds and returns all possible distributions of the valid note classes for a given function

function getVoicingPermutations(func)
    voicing_permutations = []
    voicings_initial = []
    
    function_map = functions[func][1]
    double_note = functions[func][3]
    bass_note = functions[func][2][1]
    flag = nothing
    active_notes = sort!([(i, x) for (i, x) in enumerate(function_map) if x > 0], by = x -> x[2])   # [position, value]
    
    filter!(e->e != bass_note, active_notes)
        
    if size(active_notes, 1) == 3
        flag = 0       # Double note
    elseif size(active_notes, 1) < 3
        flag = 1       # Triple note
    elseif size(active_notes, 1) > 3
        flag = 2       # Four unique notes
    end
    voicing_sans_bass = copy(active_notes)
            
    if size(active_notes, 1) == 1
        push!(voicing_sans_bass, bass_note)
        push!(voicing_sans_bass, bass_note)
        push!(voicings_initial, voicing_sans_bass)
    elseif size(active_notes, 1) == 2
        for i in 1:size(double_note, 1)
            voicing_sans_bass = copy(active_notes)
            push!(voicing_sans_bass, double_note[i])
            push!(voicings_initial, voicing_sans_bass)
        end
    elseif size(active_notes, 1) == 3
        push!(voicings_initial, voicing_sans_bass)
    end
                    
    for entry in voicings_initial
        voicings = collect(permutations(entry))
        for j in voicings
            push!(voicing_permutations, j)
        end
    end
    
    voicing_permutations = unique(voicing_permutations)   
    for p in voicing_permutations
        pushfirst!(p, bass_note)
    end
    
    return voicing_permutations
end
                
#for key in keys(functions)
#    display(key)
#    display(getVoicingPermutations(key))
#end
                

        


                


getVoicingPermutations (generic function with 1 method)

In [23]:
# Generates the candidate solutions of note distribution for a given function
                
function generateCandidates(func)
    voicing_permutations = getVoicingPermutations(func)
    valid_voicings = []
    note_repository = []   # [[all C's], [all C#'s], [all D's], ..., [all B's]]
    for note_class in 1:12
        push!(note_repository, [x for x in 40:80 if x%12 == (note_class-1)])
    end
    #println(note_repository)
    
    
    for permutation in voicing_permutations
        #println(permutation)
        voicing = []
        for b in note_repository[permutation[1][1]]
            if b in 40:60
                for t in note_repository[permutation[2][1]]
                    if t in 47:68 && t >= b && t-b <= 20
                        for a in note_repository[permutation[3][1]]
                            if a in 52:71 && a >= t && a-t <= 12                                 
                                for s in note_repository[permutation[4][1]]
                                    if s in 59:80 && s >= a && s-a <= 12  
                                        voicing = []
                                        push!(voicing, (b, permutation[1][2]), (t, permutation[2][2]), (a, permutation[3][2]), (s, permutation[4][2])) 
                                        push!(valid_voicings, voicing)
                                    end                                
                                end           
                            end    
                        end
                        
                    end
                end       
            end
        end
    end
    
    return(valid_voicings)
end

notes = []
for key in keys(functions)
    for voicing in generateCandidates(key)
        push!(notes, voicing)
    end
end


Any[Any[(48, 6), (63, 1), (67, 3), (70, 5)], Any[(60, 6), (63, 1), (67, 3), (70, 5)], Any[(48, 6), (51, 1), (58, 5), (67, 3)], Any[(48, 6), (63, 1), (70, 5), (79, 3)], Any[(60, 6), (63, 1), (70, 5), (79, 3)], Any[(48, 6), (55, 3), (63, 1), (70, 5)], Any[(48, 6), (55, 3), (58, 5), (63, 1)], Any[(48, 6), (67, 3), (70, 5), (75, 1)], Any[(60, 6), (67, 3), (70, 5), (75, 1)], Any[(48, 6), (58, 5), (63, 1), (67, 3)], Any[(48, 6), (58, 5), (67, 3), (75, 1)], Any[(46, 5), (63, 1), (67, 3), (70, 5)], Any[(58, 5), (63, 1), (67, 3), (70, 5)], Any[(46, 5), (51, 1), (58, 5), (67, 3)], Any[(46, 5), (63, 1), (70, 5), (79, 3)], Any[(58, 5), (63, 1), (70, 5), (79, 3)], Any[(46, 5), (55, 3), (63, 1), (70, 5)], Any[(46, 5), (55, 3), (58, 5), (63, 1)], Any[(58, 5), (67, 3), (70, 5), (75, 1)], Any[(46, 5), (58, 5), (63, 1), (67, 3)], Any[(58, 5), (58, 5), (63, 1), (67, 3)], Any[(46, 5), (58, 5), (67, 3), (75, 1)], Any[(58, 5), (58, 5), (67, 3), (75, 1)], Any[(46, 5), (51, 1), (55, 3), (63, 1)], Any[(46, 5),

In [26]:
## Convert back to piano roll

function makePianoRoll(notes)
    println(size(notes, 1))
    pianoroll = zeros(Int8, 128, size(notes,1))
    counter = 1
    for set in notes[:]
        for voice in set
            #println(voice)
            pianoroll[voice[1], counter] = 1
        end
        counter = counter + 1
        #println(pianoroll[:,1])
    end
    #println(findall(x->x==1, pianoroll))
    empty = zeros(Int8, 1, size(notes, 1))
    new = vcat(pianoroll, empty)
    display(new)
    writedlm("output\\test.csv",  new, ',')
end

makePianoRoll(notes)

129×359 Array{Int8,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0   

359


# # NODAL

In [145]:
import Optim
SimulatedAnnealing(; neighbor = default_neighbor!,
                    T = default_temperature,
                    p = kirkpatrick)

UndefVarError: UndefVarError: default_neighbor! not defined